<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week3/part1/3_02_Ejemplo_Memoria_Largo_Plazo_LangChain_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Memoria a Largo Plazo en Agentes (LangChain + LangGraph)

Colab-ready. Cubre: amnesia, corto vs largo, 4 pilares (Storage/Extraction/Indexing/Retrieval), técnicas de almacenamiento (relacional/vector/NoSQL/híbrido), implementación en LangChain, checkpointing en LangGraph, estrategia híbrida y mejores prácticas.


## 0) Setup


In [ ]:

!pip -q install "langchain>=1,<2" langchain-core langchain-community
!pip -q install langchain-groq langsmith chromadb langgraph pydantic


In [ ]:

import os
from google.colab import userdata

if not os.getenv("GROQ_API_KEY"):
    try:
        os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
    except Exception:
        pass

print("GROQ_API_KEY:", "✅" if os.getenv("GROQ_API_KEY") else "⚠️")

In [ ]:

from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.2)
print("✅ LLM listo:", llm.model_name)


## 1) El problema de la amnesia (sin memoria persistente)


In [ ]:

from langchain_core.messages import SystemMessage, HumanMessage

def one_shot(user_text: str):
    return llm.invoke([SystemMessage(content="Eres un asistente."), HumanMessage(content=user_text)]).content

print("Lunes:", one_shot("Me llamo Carlos y trabajo en marketing digital."))
print("Viernes:", one_shot("¿Qué cursos me recomiendas para mi profesión?"))


## 2) Corto vs Largo plazo

- **Corto**: historial de la sesión actual.
- **Largo**: hechos persistentes + recuerdos recuperables (días/semanas).


## 3) Los 4 pilares: Storage, Extraction, Indexing, Retrieval


## - Storage (hechos estructurados) con SQLite


In [ ]:

import sqlite3, time, json

conn = sqlite3.connect("memoria_struct.db")
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS user_facts (
  user_id TEXT,
  key TEXT,
  value TEXT,
  ts INTEGER,
  PRIMARY KEY (user_id, key)
)
""")
conn.commit()

def upsert_fact(user_id: str, key: str, value: str):
    cur.execute(
        "INSERT OR REPLACE INTO user_facts(user_id,key,value,ts) VALUES (?,?,?,?)",
        (user_id, key, value, int(time.time()))
    )
    conn.commit()

def get_facts(user_id: str):
    rows = cur.execute("SELECT key, value, ts FROM user_facts WHERE user_id=?", (user_id,)).fetchall()
    return {k: v for (k, v, _) in rows}

upsert_fact("carlos", "nombre", "Carlos")
upsert_fact("carlos", "profesion", "marketing digital")
print(get_facts("carlos"))


## - Storage + Indexing (memoria semántica) con Vector Store (Chroma)

In [ ]:
%pip -q install sentence-transformers

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True},
)

vectordb = Chroma(
    collection_name="memoria_conversaciones",
    embedding_function=emb,
    persist_directory=None  # in-memory
)

vectordb.add_documents([
    Document(page_content="Carlos quiere mejorar SEO.", metadata={"user_id":"carlos","ts":1}),
    Document(page_content="Carlos prefiere cursos con proyectos prácticos.", metadata={"user_id":"carlos","ts":2}),
    Document(page_content="Carlos está interesado en analítica web.", metadata={"user_id":"carlos","ts":3}),
])

print("✅ Docs indexados:", vectordb._collection.count())


## - Retrieval: traer recuerdos relevantes (filtrando por usuario)


In [ ]:

q = "¿Qué le interesa aprender a Carlos?"
hits = vectordb.similarity_search(q, k=2, filter={"user_id":"carlos"})
for d in hits:
    print("-", d.page_content)


## - Extraction: extraer hechos valiosos con salida estructurada


In [ ]:

from pydantic import BaseModel, Field
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate

class ExtractedFacts(BaseModel):
    nombre: Optional[str] = Field(default=None)
    profesion: Optional[str] = Field(default=None)
    preferencia_aprendizaje: Optional[str] = Field(default=None)

extract_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Extrae SOLO hechos estables del usuario para memoria a largo plazo. "
     "No incluyas saludos ni información temporal. Devuelve SOLO JSON."),
    ("user", "Conversación:\n{conversation}")
])

extract_chain = extract_prompt | llm.with_structured_output(ExtractedFacts)

conversation = """
Usuario: Hola, soy Carlos. Trabajo en marketing digital.
Usuario: Me gustan cursos con proyectos prácticos y casos reales.
Usuario: Esta semana estoy estresado por una entrega.
"""

facts = extract_chain.invoke({"conversation": conversation})
print(facts)


In [ ]:

for k, v in facts.model_dump().items():
    if v:
        upsert_fact("carlos", k, v)

print("Hechos persistidos:", get_facts("carlos"))


## - LangChain: responder usando hechos + recuerdos recuperados


In [ ]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import json

assistant_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente que recomienda cursos.\n"
     "Hechos (estructurados): {facts}\n"
     "Recuerdos (semánticos):\n{memories}\n"),
    ("user", "{question}")
])

def answer_with_ltm(user_id: str, question: str):
    f = get_facts(user_id)
    sem = vectordb.similarity_search(question, k=2, filter={"user_id": user_id})
    memories = "\n".join([f"- {d.page_content}" for d in sem]) if sem else "(sin recuerdos)"
    chain = assistant_prompt | llm | StrOutputParser()
    return chain.invoke({"facts": json.dumps(f, ensure_ascii=False), "memories": memories, "question": question})

print(answer_with_ltm("carlos", "¿Qué cursos me recomiendas para mi profesión?"))


## 4) LangGraph: checkpointing (memoria de sesión persistente)

Checkpointing persiste el **estado** del grafo (corto plazo entre ejecuciones).


In [ ]:

from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class ChatState(TypedDict):
    messages: List[BaseMessage]

def chat_node(state: ChatState):
    resp = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [AIMessage(content=resp.content)]}

g = StateGraph(ChatState)
g.add_node("chat", chat_node)
g.set_entry_point("chat")
g.add_edge("chat", END)

app = g.compile(checkpointer=MemorySaver())
thread_id = "thread-carlos-1"

out1 = app.invoke({"messages":[HumanMessage(content="Me llamo Carlos y trabajo en marketing digital.")]},
                  config={"configurable":{"thread_id": thread_id}})
print(out1["messages"][-1].content)

out2 = app.invoke({"messages":[HumanMessage(content="¿Qué cursos me recomiendas para mi profesión?")]},
                  config={"configurable":{"thread_id": thread_id}})
print(out2["messages"][-1].content)


## 5) Estrategia híbrida en LangGraph (Retrieval + Chat)

Checkpointing + almacenamiento externo de hechos + retrieval semántico.


In [ ]:

from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
import json

class HybridState(TypedDict):
    user_id: str
    messages: List[BaseMessage]
    facts: Dict[str, Any]
    memories: str

def retrieval_node(state: HybridState):
    uid = state["user_id"]
    f = get_facts(uid)
    q = next((m.content for m in reversed(state["messages"]) if isinstance(m, HumanMessage)), "")
    sem = vectordb.similarity_search(q, k=2, filter={"user_id": uid})
    mem = "\n".join([f"- {d.page_content}" for d in sem]) if sem else "(sin recuerdos)"
    return {"facts": f, "memories": mem}

def chat_node2(state: HybridState):
    system = SystemMessage(content=(
        "Usa estos datos para personalizar:\n"
        f"Hechos: {json.dumps(state['facts'], ensure_ascii=False)}\n"
        f"Memorias:\n{state['memories']}\n"
    ))
    resp = llm.invoke([system] + state["messages"])
    return {"messages": state["messages"] + [AIMessage(content=resp.content)]}

h = StateGraph(HybridState)
h.add_node("retrieve", retrieval_node)
h.add_node("chat", chat_node2)
h.set_entry_point("retrieve")
h.add_edge("retrieve","chat")
h.add_edge("chat", END)

app2 = h.compile(checkpointer=MemorySaver())

print(app2.invoke(
    {"user_id":"carlos","messages":[HumanMessage(content="Recomiéndame cursos para mi profesión.")], "facts":{}, "memories":""},
    config={"configurable":{"thread_id":"thread-carlos-2"}}
)["messages"][-1].content)
